In [54]:
import networkit as nk
import numpy as np
import pandas as pd

In [55]:
all_data = pd.read_csv("../results/time.csv", sep=";")
all_data = all_data[ all_data["Graph"].str.contains("DoubleExperiment")]
all_data

,Graph,Community computation,Nodes computation,GLR computation,Total
0,DoubleExperiment(n=10).graph,0.001963,0.002279,0.002571,0.002586
1,DoubleExperiment(n=10).graph,0.000300,0.000449,0.000647,0.000657
2,DoubleExperiment(n=10).graph,0.000347,0.000474,0.000670,0.000679
3,DoubleExperiment(n=10).graph,0.000331,0.000397,0.000535,0.000541
4,DoubleExperiment(n=10).graph,0.000245,0.000367,0.000551,0.000559
5,DoubleExperiment(n=10240).graph,0.057027,0.970550,62.593090,62.595007
6,DoubleExperiment(n=10240).graph,0.062452,0.913750,59.567496,59.569750
7,DoubleExperiment(n=10240).graph,0.058453,0.683484,58.517543,58.520577
8,DoubleExperiment(n=10240).graph,0.061110,0.724083,73.953123,73.957938
9,DoubleExperiment(n=10240).graph,0.059155,0.715544,83.525481,83.528159


In [56]:
grouped = all_data.groupby("Graph")
all_mean = grouped.aggregate("mean")
all_mean

,Community computation,Nodes computation,GLR computation,Total
Graph,,,,
DoubleExperiment(n=10).graph,0.000637,0.000793,0.000995,0.001004
DoubleExperiment(n=10240).graph,0.059639,0.801482,67.631347,67.634286
DoubleExperiment(n=1280).graph,0.007342,0.017594,0.886662,0.887272
DoubleExperiment(n=160).graph,0.000929,0.001599,0.022204,0.022289
DoubleExperiment(n=20).graph,0.000272,0.000385,0.000845,0.000858
DoubleExperiment(n=20480).graph,0.097486,6.744038,411.466876,411.471703
DoubleExperiment(n=2560).graph,0.011500,0.079689,4.063947,4.064856
DoubleExperiment(n=320).graph,0.001492,0.002866,0.075235,0.075378
DoubleExperiment(n=40).graph,0.000440,0.000677,0.002952,0.002976


In [57]:
import re


all_mean.index

Index(['DoubleExperiment(n=10).graph', 'DoubleExperiment(n=10240).graph',
       'DoubleExperiment(n=1280).graph', 'DoubleExperiment(n=160).graph',
       'DoubleExperiment(n=20).graph', 'DoubleExperiment(n=20480).graph',
       'DoubleExperiment(n=2560).graph', 'DoubleExperiment(n=320).graph',
       'DoubleExperiment(n=40).graph', 'DoubleExperiment(n=5120).graph',
       'DoubleExperiment(n=640).graph', 'DoubleExperiment(n=80).graph'],
      dtype='object', name='Graph')